In [1]:
import sys
sys.version

'3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]'

In [2]:
! pip install -q python-dotenv
! pip install -q neo4j
! pip install -q langchain


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from neo4j import GraphDatabase
import json

In [4]:
! pip install -q langchain-openai


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [6]:
import dotenv
import os

dotenv.load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("URL")
os.environ["NEO4J_USERNAME"]= os.getenv("USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("PASSWORD")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

In [7]:
path =r'..\data\aacsb\processed\SECTIONS_processed_2024-04-04_16-24-04.json'
json_file = json.load(open(path))
print(type(json_file)) #list

<class 'list'>


In [8]:
for element in json_file:
    for key,val in element.items():
        print(f'{key} :{val}')

        if type(val) is str:
            print(f'{key}, val_length: {len(val)}')

Section_Num :1
Section_Title :Strategic Managemend and Innovation
Section_Title, val_length: 35
Section_Description :strategic management encompasses entire range activities school engages fulfill mission informs school resource management . primary documentation strategic plan accredited schools expected principal artifact peer review team examine . strategic plan provides basis composition school intellectual contribution portfolio identification peer groups school aspirations . strategic plan identifies school mission strategic initiatives goals objectives tactics metrics success created input key stakeholders . ensuring financial vitality operational strategic perspective also critical part strategic management . additionally important component aacsb accredited institution school contribute meaningfully connecting business education business practice fostering promoting societal impact consistent aacsb vision . section provides standards guide business schools process meaningful s

In [ ]:
section_query = """
MERGE(s:Section {sectionNum: $sectionsIn.Section_Num})
    ON CREATE SET 
        s.title =  $sectionsIn.Section_Title,
        s.description =  $sectionsIn.Section_Description, 
        s.nodeType = 'SECTION', 
        s.nodeCat = 'AACSB',
        s.source = 'https://www.aacsb.edu/educators/accreditation/business-accreditation/aacsb-business-accreditation-standards',
        s.year = 2020
RETURN s
"""

In [ ]:
#
# Open Neo4j driver
#

# driver = GraphDatabase.driver(connectionURL, auth =(username,password))
# print(driver.verify_connectivity())

In [ ]:
#
# Close Connetion to Neo4j
#

# driver.close()
# print(driver.verify_connectivity())

In [9]:
#
# Neo4j with LangChain wrapper
# [ERROR RESOLVED w/ APOC PLUGIN ENABLED]
#

graphDB = Neo4jGraph()

In [10]:
section_1 = json_file[0]

In [11]:
print(type(section_1))

for k,v in section_1.items():
    print(k,type(v))


<class 'dict'>
Section_Num <class 'int'>
Section_Title <class 'str'>
Section_Description <class 'str'>


In [ ]:
graphDB.query(section_query, 
         params={'sectionsIn':section_1})

In [ ]:
#
# No duplicatipe nodes
#

graphDB.query("""
CREATE CONSTRAINT unique_node IF NOT EXISTS 
    FOR (s:Session) REQUIRE s.sectionNum IS UNIQUE
""")


In [ ]:
node_count = 0
for section in json_file:
    print(f"Creating `:Section ` node for section ID {section['Section_Num']}")
    graphDB.query(section_query, 
            params={
                'sectionsIn': section
            })
    node_count += 1
print(f"Created {node_count} nodes")

In [12]:
graphDB.query("""
MATCH (n)
RETURN COUNT(n)
""")

[{'COUNT(n)': 3}]

In [63]:
#
# STANDARDS NODES and child nodes
# @TODO Link by -- parentStandardNum
#

standards_query = """
MERGE(s:Standard {standardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        s.parentSectionNum = $standardsIn.Section,
        s.title =  $standardsIn.Standard_title,
        s.text =  $standardsIn.Standard_formal, 
        s.nodeType = 'STANDARD', 
        s.nodeCat = 'AACSB',
        s.source = 'https://www.aacsb.edu/educators/accreditation/business-accreditation/aacsb-business-accreditation-standards',
        s.year = 2020,
        s.definitionNodeID = NULL,
        s.basisForJudgementNodeID=NULL,
        s.supportingDocNodeID=NULL
RETURN s, ID(s) as elementId
"""

definitions_query = """
MERGE(d:Definitions {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        d.text = $standardsIn.Definitions,
        d.standardTitle =  $standardsIn.Standard_title,
        d.nodeType = 'DEFINITIONS', 
        d.nodeCat = 'AACSB'
RETURN d, ID(d) as elementId
"""

basis_query = """
MERGE(b:Basis {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        b.text = $standardsIn.Basis_for_judgement,
        b.standardTitle =  $standardsIn.Standard_title,
        b.nodeType = 'BASIS', 
        b.nodeCat = 'AACSB'
RETURN b, ID(b) as elementId
"""

docs_query = """
MERGE(d:Documentation {parentStandardNum: $standardsIn.Standard_num})
    ON CREATE SET 
        d.text = $standardsIn.Supporting_docs,
        d.standardTitle =  $standardsIn.Standard_title,
        d.nodeType = 'DOCUMENTATION', 
        d.nodeCat = 'AACSB'
RETURN d, ID(d) as elemendId
"""



In [13]:
#
# Read in Standards JSON
#

path =r'..\data\aacsb\processed\STANDARDS_2024-04-04_13-21-46.json'
json_file = json.load(open(path))
print(type(json_file)) #list

<class 'list'>


In [14]:
standard_1 = json_file[0]
print(type(standard_1))

for k,v in standard_1.items():
    print(k,type(v))

<class 'dict'>
Section <class 'int'>
Standard_num <class 'int'>
Standard_title <class 'str'>
Standard_formal <class 'str'>
Definitions <class 'str'>
Basis_for_judgement <class 'str'>
Supporting_docs <class 'str'>


In [32]:
basis = standard_1['Basis_for_judgement']
print(type(basis))
print(basis)
print(len(basis))

<class 'str'>
1.1 maintenance strategic plan school mission identifies attributes focus areas priorities indicate school positions among community business schools . mission informs strategic plan reviewed updated periodically alignment school normal planning cycle . mission statement articulated part strategic plan . strategic plan developed refined engagement key internal external stakeholders . strategic plan available review peer review team demonstrates commitment continuous improvement regular review revision key stakeholder input . school strategic plan defines communities learners school intends serve including level degree programs school offers . school also identifies non degree programs corresponding communities intends serve . plan identifies strategies maintenance high quality learner experiences including current relevant curriculum strategies recruitment retention qualified faculty . school strategic plan clearly identifies type intellectual contributions expects facult

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#
# @TODO change
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, #changed from 2000
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

In [26]:
basis_chunks = text_splitter.split_text(basis)

In [27]:
print(type(basis_chunks))
print(basis_chunks[0])
print(len(basis_chunks))

<class 'list'>
1.1 maintenance strategic plan school mission identifies attributes focus areas priorities indicate school positions among community business schools . mission informs strategic plan reviewed updated periodically alignment school normal planning cycle . mission statement articulated part strategic plan . strategic plan developed refined engagement key internal external stakeholders . strategic plan available review peer review team demonstrates commitment continuous improvement regular review
6


In [80]:


from typing import List,Dict, Any


def key_to_node_label(key: str) -> str:
    """
    Maps a key to its corresponding node label based on a predefined mapping.

    Args:
        key (str): The key to be mapped to a node label.

    Returns:
        str: The corresponding node label if the key is found in the mapping, otherwise an empty string.
    """
    key_node_map: Dict[str, str] = {
        'Standard_formal': 'STANDARD',
        'Definitions': 'DEFINITIONS',
        'Basis_for_judgement': 'BASIS',
        'Supporting_docs': 'DOCUMENTATION'
    }
    return key_node_map.get(key, '')  # Return empty string if key not found




def split_text_from_node(text: str, key: str) -> List[Dict[str, Any]]:
    """
    Splits the input text into chunks and generates metadata for each chunk.

    Args:
        text (str): The text to be split into chunks.
        key (str): The key representing the type of data.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries, where each dictionary represents a chunk
            with metadata including the chunk text, parent type, and chunk sequence ID.
    """
    chunks_with_metadata = []
    chunks = text_splitter.split_text(text)

    for i, chunk in enumerate(chunks):
        #print(f'{i} : {chunk}')
        chunks_with_metadata.append({
            'type': 'dataChunk',
            'text': chunk,
            'parentType': key,
            'chunk_sequence_id': i,
        })
    
    return chunks_with_metadata



def top_level(standard: Dict[str, Any]) -> Dict[str, Any]:
    """
    Processes top-level items from the standard dictionary.

    Args:
        standard (Dict[str, Any]): The dictionary containing top-level items.

    Returns:
        Dict[str, Any]: A dictionary where keys are top-level item keys ('Standard_formal',
            'Definitions', 'Basis_for_judgement', 'Supporting_docs'), and values are lists of
            dictionaries representing chunks with metadata for each item.
    """
    chunk_dict = {}

    for key in ['Standard_formal', 'Definitions', 'Basis_for_judgement', 'Supporting_docs']:
        #print(key)
        #print(type(key))
        item_text = standard[key]
        item_label = key_to_node_label(key)
        dict_val = split_text_from_node(item_text, item_label)
        chunk_dict[key] = dict_val

    return chunk_dict


# def text_data_from_standards(standard):
#     sub_section_chunks = []

#     for k,v in standard.items():
#         # get key label
#         label = key_to_node_label(k)
#         print(f'Processing{k}')
#         sub_section_chunks.append(split_text_from_node(v,label))
    
#     return sub_section_chunks


In [81]:
#
# testing split function
#

#result = split_text_from_node(basis)
result = top_level(json_file[0])

print(type(result))
print(result)

<class 'dict'>
{'Standard_formal': [{'type': 'dataChunk', 'text': 'standard 1 strategic planning1.1 school maintains well documented strategic plan developed robust collaborative planning process involving key stakeholder input informs school resource allocation priorities . strategic plan also articulate clear focused mission school . 1.2 school regularly monitors progress planned strategies expected outcomes communicates progress key stakeholders . part monitoring school conducts formal risk analysis plans mitigate identified major risks . 1.3 school carries', 'parentType': 'STANDARD', 'chunk_sequence_id': 0}, {'type': 'dataChunk', 'text': 'progress planned strategies expected outcomes communicates progress key stakeholders . part monitoring school conducts formal risk analysis plans mitigate identified major risks . 1.3 school carries mission embraces innovation key element continuous improvement . 1.4 school demonstrates commitment positive societal impact expressed supported focus

In [82]:
print(result['Standard_formal'][0])

{'type': 'dataChunk', 'text': 'standard 1 strategic planning1.1 school maintains well documented strategic plan developed robust collaborative planning process involving key stakeholder input informs school resource allocation priorities . strategic plan also articulate clear focused mission school . 1.2 school regularly monitors progress planned strategies expected outcomes communicates progress key stakeholders . part monitoring school conducts formal risk analysis plans mitigate identified major risks . 1.3 school carries', 'parentType': 'STANDARD', 'chunk_sequence_id': 0}


In [65]:
# I. CREATE STANDARDS
# for each standard in the json file
# create dict with chunked text data
# create the standard node w 
# save value of unique node ID
# create chunk nodes for standard
# link nodes to standard
# 
#II. CREATE CHILD NODES
# from standard and dict, create child nodes
# create subnodes of chunk data and link to new node
# link child node to standard
#

# standards_query = """
# MERGE(s:Standard {standardNum: $standardsIn.Standard_num})
#     ON CREATE SET 
#         s.parentSectionNum = $standardsIn.Section,
#         s.title =  $standardsIn.Standard_title,
#         s.text =  $standardsIn.Standard_formal, 
#         s.nodeType = 'STANDARD', 
#         s.nodeCat = 'AACSB',
#         s.source = 'https://www.aacsb.edu/educators/accreditation/business-accreditation/aacsb-business-accreditation-standards',
#         s.year = 2020,
#         s.definitionNodeID = NULL,
#         s.basisForJudgementNodeID=NULL,
#         s.supportingDocNodeID=NULL
# RETURN s
# """

# datachunk_query = """
# MERGE(c:Chunk {parentStandardNum: $standardsIn.Standard_num})
#     ON CREATE SET 
#         c.text = $chunkIn.text,
#         c.standardTitle =  $standardsIn.Standard_title,
#         c.nodeType = 'DATACHUNK',
#         c.parentType = $chunkIn.parentType,
#         c.nodeCat = 'AACSB',
#         c.chunkSeqNum=$chunkIn.chunk_sequence_id
# RETURN c
# """

datachunk_query = """
UNWIND $chunksIn AS chunk
MERGE (c:Chunk {
    parentStandardNum: $standardsIn.Standard_num,
    text: chunk.text,
    standardTitle: $standardsIn.Standard_title,
    nodeType: 'DATACHUNK',
    parentType: chunk.parentType,
    parentId: $parentId,
    nodeCat: 'AACSB',
    chunkSeqNum: chunk.chunk_sequence_id
})
RETURN c
"""

In [83]:
db_result = graphDB.query(standards_query,
              params={'standardsIn':standard_1})

In [77]:
db_result[0]['s']['nodeType']

'STANDARD'

In [84]:

graphDB.query(datachunk_query,
                params={
                    'standardsIn':standard_1,
                    'chunksIn':result['Standard_formal'],
                    'parentId':db_result[0]['elementId']
                })

[{'c': {'parentStandardNum': 1,
   'standardTitle': 'Strategic Planning',
   'nodeCat': 'AACSB',
   'text': 'standard 1 strategic planning1.1 school maintains well documented strategic plan developed robust collaborative planning process involving key stakeholder input informs school resource allocation priorities . strategic plan also articulate clear focused mission school . 1.2 school regularly monitors progress planned strategies expected outcomes communicates progress key stakeholders . part monitoring school conducts formal risk analysis plans mitigate identified major risks . 1.3 school carries',
   'chunkSeqNum': 0,
   'nodeType': 'DATACHUNK',
   'parentType': 'STANDARD',
   'parentId': 4}},
 {'c': {'parentStandardNum': 1,
   'standardTitle': 'Strategic Planning',
   'nodeCat': 'AACSB',
   'text': 'progress planned strategies expected outcomes communicates progress key stakeholders . part monitoring school conducts formal risk analysis plans mitigate identified major risks . 1.

In [85]:
# Create Links
linking_query = """
  MATCH (from_same_section:Chunk)
  WHERE from_same_section.parentId = $parentId
    AND from_same_section.parentType = $parentType
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqNum ASC
  WITH collect(from_same_section) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
  RETURN size(section_chunk_list)
"""

In [86]:
graphDB.query(linking_query, 
              params={
                  'parentId':db_result[0]['elementId'],
                  'parentType':db_result[0]['s']['nodeType']
              })

[{'size(section_chunk_list)': 2}]

In [ ]:
#
# @TODO Chunking Function
# @TODO Modofy Loop for standards

node_count = 0
for standard in json_file:
    print(f"Creating `:Standard ` node for section ID {standard['Standard_Num']}")
    graphDB.query(section_query, 
            params={
                'standardsIn': standard
            })
    node_count += 1
print(f"Created {node_count} nodes")
